## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [533]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Получили общую информацию о датафрейме (название заголовков, количество значений и типы данных). Так же заметили, что присутствуют пропущенные значения.

### Шаг 2. Предобработка данных

### Получим общую информацию

In [534]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Наблюдаем следующие моменты:
#### 1. В столбце children есть значения -1 и 20. Скорее всего закрылась там ошибка;
#### 2. В столбце days_employed количество отработанных дней представлено отрицательным значением (этого не может быть). Так же количество отработанных дней очень большое значение. Если рассматривать количество рабочих дней в году то в нем примерно 260 дней рабочих, то рабочий стаж получится примерно 240 лет (63046/260). Данную информацию нужно дополнительно уточнять у заказчика (у сотрудников кредитного отдела банка, но пока будем руководствоваться данными значениями);
#### 3. В столбце dob_years возраст по некоторым строкам равняется нулю (нужно проверить данный столбец).

### Заполним пропуски о доходах

In [535]:
#Определяем средний ежемесячный доход
sred_income=data['total_income'].mean()
print('Средний ежемесячный доход: {:.2f}'.format(sred_income))

Средний ежемесячный доход: 167422.30


### Но так как клиенты работают в разных сферах занятости, то для них свойственна разная величина дохода. Осуществляем группировку исходя из типа занятости. Данный подсчет предварительный, т.к. в последующим будем заполнять пропуски исходя из среднего значения.

In [536]:
# получаем табличку со средними значениями для каждого типа
sred_total_income = data.groupby(['income_type']).agg({'total_income' : 'mean'})
 
# выведем табличку, чтобы было понятно что там лежит
display(sred_total_income)

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,170898.309923
компаньон,202417.461462
пенсионер,137127.465690
предприниматель,499163.144947
сотрудник,161380.260488
студент,98201.625314


### Расчет показал, что наибольший средний доход у предпринимателя, наименьший у того кто в декрете и у студента

In [537]:
#Определяем средний общий трудовой стаж в днях
sred_employed=data['days_employed'].mean()
print('Средний общий трудовой стаж в днях: {:.2f}'.format(sred_employed))

Средний общий трудовой стаж в днях: 63046.50


### Аналогичные действия проводим для общего трудового стажа в днях

In [538]:
# получаем табличку со средними значениями для каждого типа
sred_days_empl = data.groupby(['income_type']).agg({'days_employed' : 'mean'})
 
# выведем табличку, чтобы было понятно что там лежит
display(sred_days_empl)

,days_employed
income_type,
безработный,366413.652744
в декрете,-3296.759962
госслужащий,-3399.896902
компаньон,-2111.524398
пенсионер,365003.491245
предприниматель,-520.848083
сотрудник,-2326.499216
студент,-578.751554


### Замечаем, кто работает имеет отрицательное значение трудового стажа, а кто не работает положительное (безработные и пенсионеры). При это в расчете закралась явная ошибка в группах безработные и пенсионеры. 

In [539]:
for inc_type in sred_total_income.index:
    data.loc[data['income_type'] == inc_type,'total_income'] = data.loc[data['income_type'] == inc_type,'total_income'].fillna(sred_total_income.loc[inc_type, 'total_income'])
 
 
#sred_total_income.index - это список с индексами в табличке sred_total_income, а индексами там, в результате группировки, записаны типы занятости

In [540]:
for inc_type in sred_days_empl.index:
    data.loc[data['income_type'] == inc_type,'days_employed'] = data.loc[data['income_type'] == inc_type,'days_employed'].fillna(sred_days_empl.loc[inc_type, 'days_employed'])
 
 
#sred_days_empl.index - это список с индексами в табличке sred_days_empl, а индексами там, в результате группировки, записаны типы занятости

### Так есть отрицательные значения, то используем метод abs(), который по сути берет модуль числа. 

In [541]:
data['days_employed'] = data['days_employed'].abs()

In [542]:
#Осуществил проверку отсутствующих значений
print(data.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Так тип данных в колонках 'total_income' и 'days_employed' вещественный, то поменям его на целочисленый. Целочисленный тип подсчета больше подходит для расчета.

In [543]:
data['total_income']=data['total_income'].astype('int')
data['days_employed']=data['days_employed'].astype('int')

In [544]:
#Проверим и затем скроем значения полученные
#data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Заполним явные ошибки

#### По возрасту клиента

In [545]:
sred_dob_years=data['dob_years'].mean()
print('Средний ежемесячный доход:', int(sred_dob_years))
#Средний возраст клиента равен 43 года 

Средний ежемесячный доход: 43


In [546]:
data['dob_years'] = data['dob_years'].replace(0, 43)

#### По количеству детей

In [547]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

#### Столбец education приводим к однотипным значениям

In [548]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [549]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [550]:
#Проверили типы данных
#data.info()

### Обработка дубликатов

#### Посчитаем количество дубликатов используя метод duplicated

In [551]:
data.duplicated().sum()

71

#### Посмотрим на них

In [552]:
data[data.duplicated(keep=False)].sort_values(by=['total_income','days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1005,0,365003,62,среднее,1,женат / замужем,0,F,пенсионер,0,137127,ремонт жилью
1191,0,365003,61,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с недвижимостью
1511,0,365003,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,137127,дополнительное образование
1681,0,365003,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
2052,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
16148,0,2111,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
17379,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью
17774,1,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,202417,строительство жилой недвижимости
19369,0,2111,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба


#### Похоже задвоение данных (создались как-то автоматически дубли данных). Удаляем их. 

In [553]:
data=data.drop_duplicates().reset_index(drop=True)

#### Проверяем повторно

In [554]:
data.duplicated().sum()

0

### Категоризация данных

In [555]:
status_debt=data.groupby(['income_type', 'debt']).agg({'debt':'count'})
#status_debt

In [556]:
def status_debt (data):
    debt=data['debt']
    if debt==1:
        return 'Имеется задолженность'
    return 'Задолженности нет'
data['status_debt']=data.apply(status_debt, axis=1)

In [557]:
def kids_dept (data):
    children=data['children']
    if children < 1:
        return 'нет детей'
    return 'есть дети'
data['children']=data.apply(kids_dept, axis=1) 

In [558]:
data_children_status_pivot=data.pivot_table(index=['children'],columns='status_debt',values='debt', aggfunc='count',fill_value=0, margins=True)
data_children_status_pivot['Доля'] = (data_children_status_pivot['Имеется задолженность']/data_children_status_pivot['All'])
data_children_status_pivot['Доля'] = data_children_status_pivot['Доля'].apply(lambda x: '{:.2%}'.format(x))

In [559]:
data_children_status_pivot.reset_index()

status_debt,children,Задолженности нет,Имеется задолженность,All,Доля
0,есть дети,6685,678,7363,9.21%
1,нет детей,13028,1063,14091,7.54%
2,All,19713,1741,21454,8.12%


In [560]:
data_family_status_pivot=data.pivot_table(index=['family_status'],columns='status_debt',values='debt', aggfunc='count',fill_value=0, margins=True)
data_family_status_pivot['Доля'] = (data_family_status_pivot['Имеется задолженность']/data_family_status_pivot['All'])
data_family_status_pivot['Доля'] = data_family_status_pivot['Доля'].apply(lambda x: '{:.2%}'.format(x))

In [561]:
data_family_status_pivot.reset_index()

status_debt,family_status,Задолженности нет,Имеется задолженность,All,Доля
0,Не женат / не замужем,2536,274,2810,9.75%
1,в разводе,1110,85,1195,7.11%
2,вдовец / вдова,896,63,959,6.57%
3,гражданский брак,3763,388,4151,9.35%
4,женат / замужем,11408,931,12339,7.55%
5,All,19713,1741,21454,8.12%


Женатые (замужние) клиентов процент невозврата кредита меньше чему у не женатых (не замужних) меньше на 2,2%, а по сравнению с гражданским браком - на 1,8%. При том, что кредитов у женатых больше. Ниже процент невозврата только у клиентов в разводе и у вдовцов (вдова), но и количество данных кредитов меньше примерно в 10 раз.

In [562]:
data_children_family_status_pivot=data.pivot_table(index=['children','family_status'],columns='status_debt',values='debt', aggfunc='count',fill_value=0, margins=True)

In [563]:
data_children_family_status_pivot['Доля'] = (data_children_family_status_pivot['Имеется задолженность']/data_children_family_status_pivot['All'])

In [564]:
#1ый метод
#data_children_family_status_pivot['Доля'] = data_children_family_status_pivot['Доля'].apply(lambda x: '{:.2%}'.format(x))

In [565]:
#2ый метод
def status_debt(value):
    return '{:.2%}'.format(value)
data_children_family_status_pivot['Доля'] = data_children_family_status_pivot['Доля'].apply(status_debt)

Клиенты без детей берут чаще кредиты и при этом процент невозврата у них меньше.

In [566]:
data_children_family_status_pivot.reset_index()

status_debt,children,family_status,Задолженности нет,Имеется задолженность,All,Доля
0,есть дети,Не женат / не замужем,484,64,548,11.68%
1,есть дети,в разводе,381,30,411,7.30%
2,есть дети,вдовец / вдова,102,10,112,8.93%
3,есть дети,гражданский брак,1262,159,1421,11.19%
4,есть дети,женат / замужем,4456,415,4871,8.52%
5,нет детей,Не женат / не замужем,2052,210,2262,9.28%
6,нет детей,в разводе,729,55,784,7.02%
7,нет детей,вдовец / вдова,794,53,847,6.26%
8,нет детей,гражданский брак,2501,229,2730,8.39%
9,нет детей,женат / замужем,6952,516,7468,6.91%


Развернутый анализ в соответствии с количеством детей и семейным положением

In [567]:
data_education_status_pivot=data.pivot_table(index=['education','family_status'],columns='status_debt',values='debt',aggfunc='count',fill_value=0, margins=True)

In [568]:
data_education_status_pivot['Доля'] = (data_education_status_pivot['Имеется задолженность']/data_education_status_pivot['All'])

In [569]:
data_education_status_pivot['Доля'] = data_education_status_pivot['Доля'].apply(lambda x: '{:.2%}'.format(x))

In [570]:
data_education_status_pivot2=data.pivot_table(index=['education'],columns='status_debt',values='debt',aggfunc='count',fill_value=0, margins=True)

In [571]:
data_education_status_pivot2['Доля'] = (data_education_status_pivot2['Имеется задолженность']/data_education_status_pivot2['All'])

In [572]:
data_education_status_pivot2['Доля'] = data_education_status_pivot2['Доля'].apply(lambda x: '{:.2%}'.format(x))

In [573]:
data_education_status_pivot2.reset_index()

status_debt,education,Задолженности нет,Имеется задолженность,All,Доля
0,высшее,4972,278,5250,5.30%
1,начальное,251,31,282,10.99%
2,неоконченное высшее,676,68,744,9.14%
3,среднее,13808,1364,15172,8.99%
4,ученая степень,6,0,6,0.00%
5,All,19713,1741,21454,8.12%


Клиенты с высшим образованием имеют самый низкий процент невозврат, а с ученой степенью невозвраты вообще отсутствуют.

In [574]:
data_education_status_pivot.reset_index()

status_debt,education,family_status,Задолженности нет,Имеется задолженность,All,Доля
0,высшее,Не женат / не замужем,759,46,805,5.71%
1,высшее,в разводе,301,15,316,4.75%
2,высшее,вдовец / вдова,129,7,136,5.15%
3,высшее,гражданский брак,898,55,953,5.77%
4,высшее,женат / замужем,2885,155,3040,5.10%
5,начальное,Не женат / не замужем,31,3,34,8.82%
6,начальное,в разводе,18,2,20,10.00%
7,начальное,вдовец / вдова,31,0,31,0.00%
8,начальное,гражданский брак,45,11,56,19.64%
9,начальное,женат / замужем,126,15,141,10.64%


In [575]:
def total_income_debt (data):
    if data['total_income'] <= 30000:
        return 'низкий доход'
    elif 30000 < data['total_income'] <= 80000:
        return 'средний доход'
    elif 80000 < data['total_income'] <= 150000:
        return 'высокий доход'
    else:
        return 'сверхдоход'

In [576]:
data['total_income_debt']=data.apply(total_income_debt, axis=1)
def rezyltat (categoria):
    return data.groupby(categoria)['debt'].mean().to_frame().sort_values(by='debt')
rezyltat ('total_income_debt')

,debt
total_income_debt,
средний доход,0.076309
сверхдоход,0.078984
высокий доход,0.085297
низкий доход,0.090909


In [577]:
status_income=data.groupby(['income_type', 'status_debt']).agg({'status_debt':'count','total_income':'mean'})
status_income

status_debt   total_income
income_type     status_debt                                      
безработный     Задолженности нет                1  202722.000000
                Имеется задолженность            1   59956.000000
в декрете       Имеется задолженность            1   53829.000000
госслужащий     Задолженности нет             1371  171977.783370
                Имеется задолженность           86  153681.162791
компаньон       Задолженности нет             4702  202910.361123
                Имеется задолженность          376  196246.976064
пенсионер       Задолженности нет             3613  136991.019098
                Имеется задолженность          216  139401.069444
предприниматель Задолженности нет                2  499163.000000
сотрудник       Задолженности нет            10023  161812.744089
                Имеется задолженность         1061  157289.673893
студент         Задолженности нет                1   98201.000000

Вывод: Для оценки факторов погашения кредитов осуществляем исходя из финансовых возможностей клиента (его ежемесячного дохода). Кредиты в нашей модели выдавались по следующим типам занятости, представленные выше. При этом мы наблюдаем и выделяем для себя четыре группы (сотрудник, компаньон, пенсионер и госслужащий). Три из которых имеют устойчивый и постоянный тип дохода. Как видно резкого перепада в значениях нет и носит индивидуальный характер. Это показатель платежеспособности клиента выплачивать кредит. К построению нашей модели я его б не относил, а как фактор получения кредита он будет использоваться.   

### Лемматизация

### В проекте есть столбец purpose в котором указаны цели получения кредита. В нем содержатся похожие цели, но в разно сочетании слов. Необходимо выяснить уникальные цели (леммы) и добавить их в новый столбец

In [578]:
from pymystem3 import Mystem
m=Mystem()

In [579]:
my_list=data.loc[:,'purpose'].tolist()

In [580]:
text = ','.join(my_list)

In [581]:
lemmas=m.lemmatize(text)

In [582]:
#print(lemmas)

In [583]:
from collections import Counter

### Выводим список (рейтинг) уникальных целей

In [584]:
print(Counter(lemmas))

Counter({' ': 33570, ',': 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [585]:
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)
text=Counter(data['purpose_lemma'].sum())
text.most_common()

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In [586]:
def purpose_lemma (data):
    data=data['purpose_lemma']
    try:
        if ('жилье' in data) or ('недвижимость' in data):
            return 'недвижимость'
        if 'автомобиль' in data:
            return 'автомобиль'
        if 'образование' in data:
            return 'образование'
        if 'свадьба' in data:
            return 'свадьба'
        else:
            return 'нет данных'
    except:
        return 'ошибка'

In [587]:
data['purpose_lemma']=data.apply(purpose_lemma, axis=1)
def rezyltat (categoria):
    return data.groupby(categoria)['debt'].mean().to_frame().sort_values(by='debt')
rezyltat ('purpose_lemma')

,debt
purpose_lemma,
недвижимость,0.072334
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Вывод: Клиенты без детей берут чаще кредиты и при этом процент невозврата у них меньше.
       Наблюдается устойчивая теденция влияния количества детей на возврат кредита.
       Клиенты без детей имеют большие доходы и как следствие быстрее возвратить кредит.
       В нашем случае можно присвоить баллы для оценки данной модели следующего значения:
       Нет детей           - 100б.
       Один ребенок        - 50б.
       Два ребенка         - 40б.
       В остальных случаях - 20б.  

### Вывод

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вывод: Женатые (замужние) клиентов процент невозврата кредита меньше чему у не женатых (не замужних) 
       меньше на 2,2%, а по сравнению с гражданским браком - на 1,8%. При том, что кредитов у женатых 
       больше. Ниже процент невозврата только у клиентов в разводе и у вдовцов (вдова), но и 
       количество данных кредитов меньше примерно в 10 раз.

### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

### У клиентов со средним доходом самый низкий процент невозврата кредитов (7,6%), затем идет клиенты со сверхдоходом (7,9%), потом с высоким доходом (8,5%) и самый большой невозврат кредитов у клиентов с низким доходом (9,1%). В целом мы получили общепризнанную статистику:
### Клиенты со средним доходом из большего стараются отдать кредит в оговоренные банком сроки
### Клиенты со сверхдоходом просчитывают свое финансовое состояние и,  следовательно, планируют возвраты кредита вовремя
### С высоким и низким доходом – это две самые рискованные группы. 

### Вывод

- Как разные цели кредита влияют на его возврат в срок?

### Мы получили четыре основные цели получения кредита - это недвижимость (в том числе жилье), приобретение автомобиля, образование и свадьба. На цели кредита образование и покупка автомобиля присутствует самый большой невозврат кредита и составляет чуть больше 9%.

### Шаг 4. Общий вывод

## Самые надежные клиенты банка - это женатые/замужние (или в гражданском браке, или в разводе) с ученой степенью (или с высшим образованием), конечно без детей и цель получения кредита операции с недвижимостью. 